# Introducción

En el siguiente proyecto trataré de dar solución al problema del banco Beta Bank el cual consiste en la preocupante perdida de clientes conforme pasa el tiempo, para esto se nos pide crear un modelo que pueda predecir los clientes del banco que estan más propensos a salirse para implementar un método de retención y lograr que estos clientes no abandonen el banco.

Nuestro objetivo en el modelo es lograr el máximo valor F1 posible, teniendo como mínimo un valor de al menos 0.59 con la base de datos proporcionada del banco Beta Bank.

# Importación de Librerías y Datos

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('/datasets/Churn.csv')

# Exploración Inicial de Datos

En la siguiente sección luego de haber cargado las librerías necesarias y de haber importado los datos del archivo csv, porocederé a hacer un análisis exploratorio inicial para hacer un sondeo de la base de datos y ver si todo es correcto para proceder con la creación del modelo.

In [3]:
df.info()
df.describe()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


# Eliminación de Columnas Descartables

Después de hacer un análisis inicial pude observar que para nuestro enfoque en la creación de un modelo de predicción las columnas RowNumber, CustomerId y Surname a pesar de ser numéricas no aportan ningún beneficio al modelado e incluso a mi punto de ver pueden causar alguna confusión, por eso en el siguiente paso las eliminaré, además pude observar que existen algunos valores ausentes los cuales manejaré más adelante.

In [4]:
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


# Manejo de Datos Faltantes

Al hacer el análisis exploratorio de datos me di cuenta que la variable "Tenure" tenía valores ausentes y para poder continuar con la creación del modelo debo arreglar esta situación primero, esto se puede hacer a través de distintos métodos como completar con el valor promedio, completar con el valor mediano, rellenar en función de otras variables o rellenar con valor 0 qe en este caso es el método que decidí escoger para solucionar este problema.

Escogí rellenar con el valor 0 los valores ausentes porque esto indica que los clientes con ese valor aun no han hecho un año en el banco, es decir como si fueran clientes nuevos al igual que otra parte de la base de datos en donde sí vienen indicados varios clientes con 0 que dan a entender que son nuevos y de esta manera en mi opinión, no se perjuduca tanto a las estadísticas originales que nos aporta la base de datos, ya que considero que los otros metódos son mas agresivos y podrían causar alguna distorción.

In [5]:
df['Tenure'].fillna(0, inplace=True)

df['Tenure'].value_counts().sort_index(ascending=True)

0.0     1291
1.0      952
2.0      950
3.0      928
4.0      885
5.0      927
6.0      881
7.0      925
8.0      933
9.0      882
10.0     446
Name: Tenure, dtype: int64

In [6]:
df.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

# Variables Categóricas

En la siguinte parte del proyecto voy a analizar las variables categóricas para ver de que manera las puedo manejar y continuar con la creación del modelo.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  float64
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


In [8]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Después de analisar observé que las columnas Gender y Geography son las únicas que se pueden considerar como categóricas por lo cual utilizaré el código preprocessing de la librería sklearn para transformar su contenido en valores numéricos.


In [9]:
for col in df.select_dtypes(include=['object']).columns:
    
    print(f"{col}: {df[col].unique()}")

    label_encoder = preprocessing.LabelEncoder()

    label_encoder.fit(df[col].unique())
 
    df[col] = label_encoder.transform(df[col])

    print(f"{col}: {df[col].unique()}")

Geography: ['France' 'Spain' 'Germany']
Geography: [0 2 1]
Gender: ['Female' 'Male']
Gender: [0 1]


In [10]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2.0,0.00,1,1,1,101348.88,1
1,608,2,0,41,1.0,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8.0,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1.0,0.00,2,0,0,93826.63,0
4,850,2,0,43,2.0,125510.82,1,1,1,79084.10,0


# División en Conjuntos de Entrenamiento y de Prueba

En esta sección dividiré los datos en conjuntos de entrenamiento y prueba.

In [11]:
train_ratio = 0.75
val_ratio = 0.15 
test_ratio = 0.15

train_data, test_data = train_test_split(df, test_size=test_ratio, random_state=42)

train_data, val_data = train_test_split(train_data, test_size=val_ratio/(train_ratio+val_ratio), random_state=42)
print("Tamaño del conjunto de entrenamiento:", len(train_data))
print("Tamaño del conjunto de prueba:", len(test_data))
print("Tamaño del conjunto de validación:", len(val_data))

Tamaño del conjunto de entrenamiento: 7083
Tamaño del conjunto de prueba: 1500
Tamaño del conjunto de validación: 1417


In [12]:
features_train = train_data.drop("Exited", axis=1)
target_train = train_data["Exited"]

features_test = test_data.drop("Exited", axis=1)
target_test = test_data["Exited"]

features_valid = val_data.drop("Exited", axis=1)
target_valid = val_data["Exited"]

# Análisis del Equilibrio de Clases

En esta sección haré un análisis del equilibrio de las clases en el conjunto de entrenamiento y validación con el objetivo de verificar si hay un desequilibrio significativo entre las clases positivas que son los clientes que abandonaron el banco y las clases negativas que son los clientes que permanecieron en el banco.

Con este análisis se tendrá una comprensión inicial de la distribución de los datos y así poder tomar decisiones más precisas al construir y evaluar el modelo.

In [13]:
print("Equilibrio de las clases en el conjunto de entrenamiento:")
print(target_train.value_counts(normalize=True))

print("Equilibrio de las clases en el conjunto de validación:")
print(target_valid.value_counts(normalize=True))

Equilibrio de las clases en el conjunto de entrenamiento:
0    0.796979
1    0.203021
Name: Exited, dtype: float64
Equilibrio de las clases en el conjunto de validación:
0    0.784051
1    0.215949
Name: Exited, dtype: float64


Después de analizar el equilibrio de las clases en el conjunto de entrenamiento, se puede observar que la proporción de clases es de aproximadamente el 79.6% para la Clase 0 que son los clientes que no abandonaron el banco y el 20.3% para la Clase 1 que son los clientes que abandonaron el banco mientras que en el conjunto de validación las proporciones son de alrededor del 78.4% para la clase 0 y el 21.5% para la Clase 1, estos resultados indican que hay un cierto desequilibrio en las clases con la clase 0 predominante en ambos conjuntos.

Esto es importante para nuestro proyecto, ya que el desequilibrio de clases puede afectar el rendimiento del modelo porque  puede ser sesgado a favor de la clase mayoritaria por lo cual trataré con ello más adelante para evitar problemas en el modelo


# Construcción del Modelo Inicial (sin considerar el desequilibrio de clases)

In [14]:
model = LogisticRegression()

model.fit(features_train, target_train)

predictions = model.predict(features_valid)

print(classification_report(target_valid, predictions))

              precision    recall  f1-score   support

           0       0.79      0.98      0.88      1111
           1       0.49      0.06      0.10       306

    accuracy                           0.78      1417
   macro avg       0.64      0.52      0.49      1417
weighted avg       0.72      0.78      0.71      1417



Evaluación del desempeño del modelo inicial: Al observar estos resultados se puede ver que el modelo obtuvo una alta precisión para la clase 0 que son los clientes que no se fueron y una baja precisión para la clase 1 que son los clientes que se fueron, además, el recall de la clase 1 y el valor F1 también son bajos, todo esto indica que el modelo tiene dificultades para identificar correctamente a los clientes que se han ido y por lo tanto tiene un mal rendimiento para predecir a los clientes que abandonarán el banco, en las siguientes secciones haré cambios para mejorar el modelo y obtener un buen rendimiento.

# Mejora del modelo

En esta sección me centraré en mejorar el rendimiento del modelo inicial, equilibraré los atributos numéricos para garantizar que los valores de diferentes escalas tengan la misma importancia en el modelo y tendré en cuenta el desequilibrio de clase presente en los datos aplicando diferentes técnicas para corregir este desequilibrio y así poder evaluar el impacto de estos enfoques en el rendimiento general del modelo.

In [15]:
numeric=['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])
features_test.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
6252,-0.575693,1,1,-0.661804,-0.494656,0.335315,0.805116,0,0,-1.011739
4684,-0.296070,0,1,0.379897,-1.137115,-1.213746,0.805116,1,1,0.803046
1731,-0.523911,2,0,0.474597,-0.173427,-1.213746,0.805116,1,0,-0.720708
4742,-1.507767,1,1,1.895098,1.111491,0.694801,0.805116,1,1,1.224689
4521,-0.948522,2,0,-1.135305,0.790262,0.788407,-0.923480,1,1,0.252846


# Corrección del Desequilibrio de Clases

En esta sección aplicaré las técnicas de sobremuestreo y la submuestreo para corregir el desequilibrio de clases ya que estas nos ayudarán a equilibrar la representación de las clases minoritarias y mayoritarias mejorando la capacidad del modelo para aprender de manera efectiva de los datos disponibles.

# Sobremuestreo

In [16]:
train_data = pd.concat([features_train, target_train], axis=1)

train_data_majority = train_data[train_data['Exited'] == 0]
train_data_minority = train_data[train_data['Exited'] == 1]

train_data_minority_oversampled = train_data_minority.sample(n=len(train_data_majority), replace=True, random_state=42)

train_data_combined = pd.concat([train_data_majority, train_data_minority_oversampled])

features_train_oversampled = train_data_combined.drop("Exited", axis=1)
target_train_oversampled = train_data_combined["Exited"]

print("Equilibrio de clases después del sobremuestreo:")
print(target_train_oversampled.value_counts(normalize=True))

Equilibrio de clases después del sobremuestreo:
0    0.5
1    0.5
Name: Exited, dtype: float64


In [17]:
model_oversampled = LogisticRegression()
model_oversampled.fit(features_train_oversampled, target_train_oversampled)
predictions_oversampled = model_oversampled.predict(features_valid)

print("Informe de clasificación después del sobremuestreo:")
print(classification_report(target_valid, predictions_oversampled))

Informe de clasificación después del sobremuestreo:
              precision    recall  f1-score   support

           0       0.89      0.70      0.79      1111
           1       0.39      0.69      0.50       306

    accuracy                           0.70      1417
   macro avg       0.64      0.70      0.64      1417
weighted avg       0.78      0.70      0.72      1417



Al comparar los resultados de antes y después del sobremuestreo se puede observar una mejora en las métricas de recall y F1 para la clase minoritaria que en este caso es la clase 1, lo que indica que el modelo puede identificar mejor los ejemplos de esta clase pero como contra podemos ver que la precisión del modelo disminuyó después de aplicar el sobremuestreo, sin embargo, a pesar de esto se puede decir que hubo una mejora pero probaré en el siguiente bloque con el submuestreo para ver si arroja aún mejores resultados.

# Submuestreo

In [18]:
train_data = pd.concat([features_train, target_train], axis=1)

train_data_majority = train_data[train_data['Exited'] == 0]
train_data_minority = train_data[train_data['Exited'] == 1]

train_data_majority_undersampled = train_data_majority.sample(n=len(train_data_minority), random_state=42)

train_data_combined = pd.concat([train_data_majority_undersampled, train_data_minority])

features_train_undersampled = train_data_combined.drop("Exited", axis=1)
target_train_undersampled = train_data_combined["Exited"]

print("Equilibrio de clases después del submuestreo:")
print(target_train_undersampled.value_counts(normalize=True))

Equilibrio de clases después del submuestreo:
0    0.5
1    0.5
Name: Exited, dtype: float64


In [19]:
model_undersampled = LogisticRegression()
model_undersampled.fit(features_train_undersampled, target_train_undersampled)

predictions_undersampled = model_undersampled.predict(features_valid)

report_undersampled = classification_report(target_valid, predictions_undersampled)
print("Informe de clasificación después del submuestreo:")
print(report_undersampled)

Informe de clasificación después del submuestreo:
              precision    recall  f1-score   support

           0       0.90      0.71      0.79      1111
           1       0.40      0.70      0.51       306

    accuracy                           0.71      1417
   macro avg       0.65      0.71      0.65      1417
weighted avg       0.79      0.71      0.73      1417



Después de realizar el submuestreo se puede observar que las estadísticas son aún mejores con esta técnica que con la de sobremuestreo, aunque ambas mejoran el modelo inicial sin considerar el desequilibrio de clases, el modelo de submuestreo es el que arroja mejores resultados.

# Selección del Mejor Modelo y Parámetros

Después de abordar el desequilibrio de clases, trataré de seleccionar el mejor modelo y los mejores parámetros al intentar diferentes modelos como regresión logística, árbol de decisión y random forest, además, ajustaré sus parámetros para obtener el mejor rendimiento posible.

# Regresión Logística

In [20]:
param_grid = {
     'C': [0.1, 1, 10],
     'penalty': ['l1', 'l2'],
     'solver': ['liblinear'],
     'class_weight': ['balanced']
}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='f1')

grid_search.fit(features_train_undersampled, target_train_undersampled)

best_model_lr = grid_search.best_estimator_
best_params_lr = grid_search.best_params_

best_model_lr.fit(features_train_undersampled, target_train_undersampled)

predictions_best_lr = best_model_lr.predict(features_valid)

report_best_lr = classification_report(target_valid, predictions_best_lr)
print("Informe de clasificación del método de regresión logística:")
print(report_best_lr)

print("Mejores parámetros de regresión logística:")
print(best_params_lr)

Informe de clasificación del método de regresión logística:
              precision    recall  f1-score   support

           0       0.90      0.72      0.80      1111
           1       0.40      0.70      0.51       306

    accuracy                           0.71      1417
   macro avg       0.65      0.71      0.65      1417
weighted avg       0.79      0.71      0.74      1417

Mejores parámetros de regresión logística:
{'C': 0.1, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'liblinear'}


# Árbol de decisión

In [21]:
param_grid = {
     'max_depth': np.linspace(1, 20, 20, dtype=int),
     'class_weight': ['balanced']
}

grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5, scoring='f1')

grid_search.fit(features_train_undersampled, target_train_undersampled)

best_model_dt = grid_search.best_estimator_
best_params_dt = grid_search.best_params_

best_model_dt.fit(features_train_undersampled, target_train_undersampled)

predictions_best_dt = best_model_dt.predict(features_valid)

report_best_dt = classification_report(target_valid, predictions_best_dt)
print("Informe de clasificación del árbol de decisión:")
print(report_best_dt)

print("Los mejores parámetros del árbol de decisión:")
print(best_params_dt)

Informe de clasificación del árbol de decisión:
              precision    recall  f1-score   support

           0       0.92      0.70      0.80      1111
           1       0.42      0.77      0.54       306

    accuracy                           0.72      1417
   macro avg       0.67      0.74      0.67      1417
weighted avg       0.81      0.72      0.74      1417

Los mejores parámetros del árbol de decisión:
{'class_weight': 'balanced', 'max_depth': 6}


# Random Forest

In [22]:
for depth in range(7, 12):
        model = RandomForestClassifier(random_state=12345, max_depth=depth)
        model.fit(features_train,target_train)
        
        train_predictions = model.predict(features_train)
        predictions_valid = model.predict(features_valid)
        
        modelb = RandomForestClassifier(random_state=12345, max_depth=depth, class_weight='balanced')
        modelb.fit(features_train,target_train)
        
        train_predictionsb = modelb.predict(features_train)
        predictions_validb = modelb.predict(features_valid)
        
        print( 'depth', depth, ": ")
        print('train:',accuracy_score(target_train, train_predictions))
        print('valid:',accuracy_score(target_valid, predictions_valid))
        print('trainB:',accuracy_score(target_train, train_predictionsb))
        print('validB:',accuracy_score(target_valid, predictions_validb))
        print()

depth 7 : 
train: 0.8661584074544685
valid: 0.858151023288638
trainB: 0.8341098404630806
validB: 0.8009880028228652

depth 8 : 
train: 0.8749117605534378
valid: 0.8652081863091038
trainB: 0.8566991387830015
validB: 0.8165137614678899

depth 9 : 
train: 0.8850769447974022
valid: 0.8645024700070572
trainB: 0.8831003811944091
validB: 0.8193366266760762

depth 10 : 
train: 0.9008894536213469
valid: 0.8659139026111503
trainB: 0.9114781872088098
validB: 0.8390966831333804

depth 11 : 
train: 0.9167019624452916
valid: 0.8715596330275229
trainB: 0.9442326697726952
validB: 0.8461538461538461



Prueba del modelo de random forest en el conjunto de sobremuestreo

In [23]:
best_model_rf_oversample = RandomForestClassifier(random_state=12345, max_depth=10, class_weight='balanced')
best_model_rf_oversample.fit(features_train_oversampled, target_train_oversampled)

predicted_valid = best_model_rf_oversample.predict(features_valid)

report_best_rf = classification_report(target_valid, predicted_valid)
print("Informe de clasificación de random forest sobremuestreado:")
print(report_best_rf)

print("Los mejores parámetros de random forest sobremuestreado:")
print(best_model_rf_oversample)
print()
print("Resultado de la puntuación F1 en random forest sobremuestreado:")
print(f1_score(target_valid,predicted_valid))

Informe de clasificación de random forest sobremuestreado:
              precision    recall  f1-score   support

           0       0.91      0.86      0.89      1111
           1       0.58      0.69      0.63       306

    accuracy                           0.83      1417
   macro avg       0.75      0.78      0.76      1417
weighted avg       0.84      0.83      0.83      1417

Los mejores parámetros de random forest sobremuestreado:
RandomForestClassifier(class_weight='balanced', max_depth=10,
                       random_state=12345)

Resultado de la puntuación F1 en random forest sobremuestreado:
0.6326836581709145


Prueba del modelo Random Forest en el conjunto de submuestreo

In [24]:
best_model_rf_undersample = RandomForestClassifier(random_state=12345, max_depth=10, class_weight='balanced')
best_model_rf_undersample.fit(features_train_undersampled, target_train_undersampled)

predicted_valid = best_model_rf_undersample.predict(features_valid)

report_best_rf = classification_report(target_valid, predicted_valid)
print("Informe de clasificación de random forest submuestreado")
print(report_best_rf)

print("Los mejores parámetros de random forest submuestreado:")
print(best_model_rf_undersample)
print()
print("Resultado de la puntuación F1 en random forest submuestreado:")
print(f1_score(target_valid,predicted_valid))

Informe de clasificación de random forest submuestreado
              precision    recall  f1-score   support

           0       0.92      0.78      0.84      1111
           1       0.49      0.77      0.60       306

    accuracy                           0.77      1417
   macro avg       0.71      0.77      0.72      1417
weighted avg       0.83      0.77      0.79      1417

Los mejores parámetros de random forest submuestreado:
RandomForestClassifier(class_weight='balanced', max_depth=10,
                       random_state=12345)

Resultado de la puntuación F1 en random forest submuestreado:
0.5967130214917825


Prueba del modelo Random Forest en el conjunto desequilibrado

In [25]:
best_model_rf_unbalanced = RandomForestClassifier(random_state=12345, max_depth=10, class_weight='balanced')
best_model_rf_unbalanced.fit(features_train, target_train)

predicted_valid = best_model_rf_unbalanced.predict(features_valid)

report_best_rf = classification_report(target_valid, predicted_valid)
print("Informe de clasificación de random forest desequilibrado")
print(report_best_rf)

print("Los mejores parámetros de random forest desequilibrado:")
print(best_model_rf_unbalanced)
print()
print("Resultado de la puntuación F1 en random forest desequilibrado:")
print(f1_score(target_valid,predicted_valid))

Informe de clasificación de random forest desequilibrado
              precision    recall  f1-score   support

           0       0.90      0.89      0.90      1111
           1       0.62      0.64      0.63       306

    accuracy                           0.84      1417
   macro avg       0.76      0.77      0.77      1417
weighted avg       0.84      0.84      0.84      1417

Los mejores parámetros de random forest desequilibrado:
RandomForestClassifier(class_weight='balanced', max_depth=10,
                       random_state=12345)

Resultado de la puntuación F1 en random forest desequilibrado:
0.6334405144694534


Después de realizar varios intentos con diferentes modelos y despúes de analizar los resultados de cada uno, considero que el modelo que obtuvo mejores resultados fue el modelo de random forest en el conjunto desequilibrado dado que resulto un valor F1 de 0.63 y una precisión del 0.84 aproximadamente, por eso escogeré este modelo para continuar con el proceso.

# Evaluación del Rendimiento del Modelo

Ahora evaluaremos el rendimiento del modelo utilizando métricas como precisión, recall, F1 y AUC-ROC y compararemos estas métricas con las obtenidas por el modelo anterior para evaluar si hay mejoría.

In [26]:
predictions_best = best_model_rf_unbalanced.predict(features_valid)

report_best = classification_report(target_valid, predictions_best, output_dict=True)
auc_roc_best = roc_auc_score(target_valid, predictions_best)

print("Informe de clasificación del mejor modelo:")
print(classification_report(target_valid, predictions_best))
print("AUC-ROC del mejor modelo: {:.3f}".format(auc_roc_best))

report_initial = classification_report(target_valid, predictions, output_dict=True)
auc_roc_initial = roc_auc_score(target_valid, predictions)

print("\Informe de calificación del modelo: inicial")
print(classification_report(target_valid, predictions))
print("AUC-ROC del modelo inicial: {:.3f}".format(auc_roc_initial))

print("\nComparación de métricas de rendimiento:")
print(" Initial Model | Best Model")
print("Precision (class 0): {:10.3f} | {:10.3f}".format(report_initial["0"]["precision"], report_best["0"]["precision"]))
print("Recall (class 0): {:10.3f} | {:10.3f}".format(report_initial["0"]["recall"], report_best["0"]["recall"]))
print("F1-score (class 0): {:10.3f} | {:10.3f}".format(report_initial["0"]["f1-score"], report_best["0"]["f1-score"]))
print("Accuracy (class 1): {:10.3f} | {:10.3f}".format(report_initial["1"]["precision"], report_best["1"]["precision"]))
print("Recall (class 1): {:10.3f} | {:10.3f}".format(report_initial["1"]["recall"], report_best["1"]["recall"]))
print("F1-score (class 1): {:10.3f} | {:10.3f}".format(report_initial["1"]["f1-score"], report_best["1"]["f1-score"]))
print("AUC-ROC: {:10.3f} | {:10.3f}".format(auc_roc_initial, auc_roc_best))

Informe de clasificación del mejor modelo:
              precision    recall  f1-score   support

           0       0.90      0.89      0.90      1111
           1       0.62      0.64      0.63       306

    accuracy                           0.84      1417
   macro avg       0.76      0.77      0.77      1417
weighted avg       0.84      0.84      0.84      1417

AUC-ROC del mejor modelo: 0.768
\Informe de calificación del modelo: inicial
              precision    recall  f1-score   support

           0       0.79      0.98      0.88      1111
           1       0.49      0.06      0.10       306

    accuracy                           0.78      1417
   macro avg       0.64      0.52      0.49      1417
weighted avg       0.72      0.78      0.71      1417

AUC-ROC del modelo inicial: 0.520

Comparación de métricas de rendimiento:
 Initial Model | Best Model
Precision (class 0):      0.791 |      0.901
Recall (class 0):      0.984 |      0.893
F1-score (class 0):      0.877 |    

Después de analizar los resultados puedo decir que, en general, el  modelo mejorado tuvo un mejor rendimiento que el modelo inicial sólo hubo una mínima disminución en la recuperación de la clase 0 pero logro mejoras en todos los otros apartados incluso logró una puntuación AUC-ROC mayor por eso deduzco que el modelo mejorado es más equilibrado y capaz de ayudarnos a lograr el objetivo planteado.

# Prueba Final y Aplicación del Modelo al Conjunto de Pruebas

Para aplicar el modelo al conjunto de pruebas utilizaré la función de predicción del mejor modelo entrenado, pasando como entrada las variables independientes del conjunto de pruebas y  luego compararé las predicciones obtenidas con las clases reales del conjunto de pruebas para evaluar el desempeño del modelo.

In [27]:
predictions_test_rf = best_model_rf_unbalanced.predict(features_test)

# Evaluación del valor F1

A continuación calcularé el valor F1 para el conjunto de prueba, comparando las predicciones del modelo con las etiquetas reales y almacenando el valor F1 en una variable.

In [28]:
f1 = f1_score(target_test, predictions_test_rf)

print("F1 value: {:.3f}".format(f1))

F1 value: 0.624


# Evaluación de Métricas AUC-ROC

Ahora calcularé la métrica AUC-ROC para el conjunto de prueba comparando las predicciones del modelo con las etiquetas reales y almacenando la puntuación AUC-ROC en una variable.

In [29]:
auc_roc = roc_auc_score(target_test, predictions_test_rf)

print("AUC-ROC: {:.3f}".format(auc_roc))

AUC-ROC: 0.774


# Comparación del valor F1 con el requisito mínimo y comparación de la métrica AUC-ROC con el modelo inicial

In [30]:
if f1 >= 0.59:
     print("El valor F1 cumple el requisito mínimo de 0.59.")
else:
     print("El valor F1 no cumple el requisito mínimo de 0.59.")

if auc_roc > auc_roc_initial:
     print("Se ha mejorado la métrica AUC-ROC con respecto al modelo inicial.")
else:
     print("No se ha mejorado la métrica AUC-ROC con respecto al modelo inicial.")

El valor F1 cumple el requisito mínimo de 0.59.
Se ha mejorado la métrica AUC-ROC con respecto al modelo inicial.


# Conclusión

A lo largo de este proyecto he tratado de dar solución al problema de Beta Bank con la pérdida de clientes a través del objetivo planteado de lograr un valor F1 superior a 0.59, traté de probar varios métodos para lograr crear el mejor modelo posible para la resolución del problema, pasando por la estructuración de los datos, aplicando métodos para el desequilibrio de clases y obteniendo las mejores métricas para los distintos modelos probados, al final concluí que el mejor modelo en base a los resultados que obtuve y en mi opinión era el random forest desequilibrado con el cual logré satisfacer y superar por poco el requisito principal del valor F1  además de lograr una métrica AUC-ROC mejorada.
